In [16]:
import import_ipynb

In [17]:
%run NLTK.ipynb import Tokenize,LoweringStemming,BagOfWord  # Use %run to import other jupyte notebook file

In [18]:
AllWords = []
Tags = []
XY = []

XTrain = []
YTrain = []

In [19]:
import json
with open("TestData.json", "r") as F:
    intents = json.load(F)
intents

{'Data': [{'tag': 'start',
   'patterns': ['Hi',
    'Hello',
    'Hey',
    'Namaste',
    'How are you?',
    'Is anyone there?',
    'Good day',
    'Hi there',
    'Hey there',
    "What's up?",
    'Howdy',
    'Greetings',
    'Morning!',
    'Afternoon!',
    'Evening!',
    'Hello ji',
    'Hiya!',
    'Yo!',
    'Whatâ€™s cooking?',
    'Howâ€™s it going?',
    'Nice to meet you!',
    'Kya haal hai?',
    'Sab theek?',
    'Hello bhai',
    'Kaise ho?'],
   'responses': ['Hello! How can I assist you today?',
    'Hi there! What can I do for you?',
    'Hey! How can I help?',
    'Namaste! What brings you here today?',
    'Hi! How can I make your day better?']},
  {'tag': 'stop',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Thanks for the help',
    'Take care',
    'Catch you later',
    'Talk to you soon',
    'Farewell',
    'Have a great day',
    'See you soon',
    'Later',
    'Iâ€™m off',
    'Iâ€™m out',
    'See ya',
    'Until next time',
    'Go

In [20]:
len(Tags)

0

In [21]:
for intent in intents["Data"]:
    Tag = intent["tag"]
    Tags.append(Tag)
    for Pattern in intent["patterns"]:
        Words = Tokenize(Pattern)
        Words = [LoweringStemming(Stem) for Stem in Words if(Stem not in ["?","!",".",",","'s"])]
        AllWords.extend(Words)  #Words is a list
        XY.append((Words,Tag))
        #print(Words)

AllWords = sorted(set(AllWords)) #Set() for remove duplicate
print("AllWords : ",len(AllWords))
#print("XY : ",XY)

AllWords :  123


In [22]:
import numpy as np
for (Pattern,Tag) in XY:
    XTrain.append(BagOfWord(Pattern,AllWords))
    YTrain.append(Tags.index(Tag))  
XTrain = np.array(XTrain)
YTrain = np.array(YTrain)

In [23]:
XTrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
YTrain

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])

In [25]:
from torch.utils.data import Dataset,DataLoader
class ChatDataset(Dataset):
    # def __init__(self):
    #     self.NSample = len(XTrain)
    #     self.XData = XTrain
    #     self.YData = YTrain

    # def __getitem__(self,index):
    #     return self.XData[index],self.YData[index]

    def __init__(self):
        self.NSample = len(XTrain)
        self.XData = XTrain
        self.YData = YTrain

    def __getitem__(self, index):
        return self.XData[index], self.YData[index]

    def __len__(self):
        return self.NSample

In [26]:
BatchSize = 5 #8
DataSet = ChatDataset()

TrainLoader = DataLoader(dataset=DataSet, batch_size=BatchSize, shuffle=True)
print(TrainLoader)

In [27]:
%run Model.ipynb import NeuralNet

In [28]:
InputSize = len(AllWords)
HiddenSize =  8 #15
NumClasses = len(Tags)
print(InputSize, HiddenSize, NumClasses)

Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model = NeuralNet(InputSize, HiddenSize, NumClasses).to(Device)
print(Device)

123 8 4
cuda


In [29]:

LearningRate = 0.0001 #0.00001
NumEpochs = 1000

Criterian = nn.CrossEntropyLoss()
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

for Epochs in range(NumEpochs):
    for (Words,Labels) in TrainLoader:
        Words = Words.to(Device)
        Labels = Labels.to(Device).long()    

        Output = Model(Words)
        Loss = Criterian(Output, Labels)

        Optimizer.zero_grad()
        Loss.backward()
        Optimizer.step()

    if(Epochs + 1) % 100 == 0 :
        print(f'Epoch [{Epochs+1}/{NumEpochs}], Loss: {Loss.item():.4f}')
        
print(f"Final Loss , Loss={Loss.item():.4f}")

Epoch [100/1000], Loss: 1.3121
Epoch [200/1000], Loss: 0.6669
Epoch [300/1000], Loss: 0.6256
Epoch [400/1000], Loss: 0.1600
Epoch [500/1000], Loss: 0.0241
Epoch [600/1000], Loss: 0.0249
Epoch [700/1000], Loss: 0.0137
Epoch [800/1000], Loss: 0.0054
Epoch [900/1000], Loss: 0.0004
Epoch [1000/1000], Loss: 0.0242
Final Loss , Loss=0.0242


In [30]:
data = {
"model_state": Model.state_dict(),
"input_size": InputSize,
"hidden_size": HiddenSize,
"output_size": NumClasses,
"all_words": AllWords,
"tags": Tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
